<img src="../../Img/backdrop-wh.png" alt="Drawing" style="width: 300px;"/>

DIGHUM160 - Critical Digital Humanities<br>
Digital Hermeneutics<br>
OPTIONAL: Relational Data <br>
Created by Tom van Nuenen (tom.van_nuenen@kcl.ac.uk)

## Relational Data

When doing research on subreddits, we might want to concatenate the associated submissions and comments (i.e., the threads), as we can assume they are about the same topic. These are saved in two different DataFrames, which are related through unique identifiers.

So let's grab the posts and comments from our subreddit.

In [1]:
import os

# We include two ../ because we want to go two levels up in the file structure
os.chdir("../../Data")

In [2]:
import pandas as pd

df_sub = pd.read_csv('aita_sub_top_sm.csv')
df_com = pd.read_csv('aita_com_top.csv')

In [4]:
df_sub.head(3)

,idint,idstr,created,self,nsfw,author,title,url,selftext,score,subreddit,distinguish,textlen,num_comments,flair_text,flair_css_class,augmented_at,augmented_count
0,427576402,t3_72kg2a,1506433689,1.0,0.0,Ritsku,AITA for breaking up with my girlfriend becaus...,NaN,My girlfriend recently went to the beach with ...,679.0,AmItheAsshole,NaN,4917.0,434.0,no a--holes here,NaN,NaN,NaN
1,551887974,t3_94kvhi,1533404095,1.0,0.0,hhhhhhffff678,AITA for banning smoking in my house and telli...,NaN,My parents smoke like chimneys. I used to as w...,832.0,AmItheAsshole,NaN,2076.0,357.0,asshole,ass,NaN,NaN
2,552654542,t3_951az2,1533562299,1.0,0.0,creepatthepool,AITA? Creep wears skimpy bathing suit to pool,NaN,Hi guys. Throwaway for obv reasons.\n\nI'm a f...,23.0,AmItheAsshole,NaN,1741.0,335.0,Shitpost,NaN,NaN,NaN


In [5]:
df_com.head(3)

,idint,idstr,created,author,parent,submission,body,score,subreddit,distinguish,textlen
0,27068149210,t1_cfnodlm,1393275432,swallowedfilth,t3_1ytr72,t3_1ytr72,Seems like you have a genuine reason to be ups...,33.0,AmItheAsshole,NaN,210.0
1,27068245493,t1_cfnqfw5,1393279437,RoonilaWazlib,t3_1ytxov,t3_1ytxov,There is nothing wrong with your original comm...,47.0,AmItheAsshole,NaN,561.0
2,27068297463,t1_cfnrjzr,1393281730,flignir,t3_1yu29c,t3_1yu29c,Yep. You're the asshole. If you want to make...,116.0,AmItheAsshole,NaN,1118.0


We should get rid of the empty cells in our two DataFrames. We can do this using the following code. 

`dropna()` gets rid of NaN values in our "selftext" column, while `isin()` checks whether each element in the DataFrame contains the values '[removed]' or '[deleted]' (which is a Reddit convention for deleted posts).

Note we need to clean up the "selftext" column for the submissions DF, and the "body" column for the comments DF (they are named differently).

In [6]:
df_sub = df_sub[~df_sub['selftext'].isin(['[removed]', '[deleted]' ])].dropna(subset=['selftext'])
df_com = df_com[~df_com['body'].isin(['[removed]', '[deleted]' ])].dropna(subset=['body'])

In [7]:
print(len(df_sub))
print(len(df_com))

16313
299877


### Concatenating submissions and comments using Pandas

Using the `pd.merge()` method, we can do an "inner join" of our two DataFrames. This is a relational database operation, which is a common operation in SQL. See [here](https://www.w3schools.com/sql/sql_join.asp) if you want to learn more. An "inner join" will yield a new DF which only contains those submissions that have associated comments (based on their "idstr" and "parent" values).

![alt text](https://s33046.pcdn.co/wp-content/uploads/2019/06/venn-diagram-representation-of-sql-inner-join-.png)

In [19]:
# merge DF based on idstr and parent
df_t = pd.merge(df_sub, df_com, how='inner', left_on='idstr', right_on='parent')

We now have a DataFrame containing all original posts / submissions, *and* the comments associated with this original post! Note that this yields a lot of rows with the same "selftext" value (as lots of comments refer to the same original post).

Also note that this operations has changed the names of the columns: we now have "x" and "y" columns based on the original DataFrame we took the data from. "idstr_x", for instance, refers to the "idstr" column from the `trp_sub` DataFrame.

In [22]:
df_t.to_csv('amita_threads.csv')

In [23]:
df_t.head()

,idint_x,idstr_x,created_x,self,nsfw,author_x,title,url,selftext,score_x,...,idstr_y,created_y,author_y,parent,submission,body,score_y,subreddit_y,distinguish_y,textlen_y
0,427576402,t3_72kg2a,1506433689,1.0,0.0,Ritsku,AITA for breaking up with my girlfriend becaus...,NaN,My girlfriend recently went to the beach with ...,679.0,...,t1_dnj53rc,1506434437,turkey3_scratch,t3_72kg2a,t3_72kg2a,I think you definitely made the right decision.,260.0,AmItheAsshole,NaN,47.0
1,427576402,t3_72kg2a,1506433689,1.0,0.0,Ritsku,AITA for breaking up with my girlfriend becaus...,NaN,My girlfriend recently went to the beach with ...,679.0,...,t1_dnj5vj0,1506435358,sweetprince686,t3_72kg2a,t3_72kg2a,I think you are being a bit controlling. You w...,93.0,AmItheAsshole,NaN,399.0
2,427576402,t3_72kg2a,1506433689,1.0,0.0,Ritsku,AITA for breaking up with my girlfriend becaus...,NaN,My girlfriend recently went to the beach with ...,679.0,...,t1_dnj60db,1506435516,insigniayellow,t3_72kg2a,t3_72kg2a,"She wasn't in the wrong to take the pictures, ...",1252.0,AmItheAsshole,NaN,861.0
3,427576402,t3_72kg2a,1506433689,1.0,0.0,Ritsku,AITA for breaking up with my girlfriend becaus...,NaN,My girlfriend recently went to the beach with ...,679.0,...,t1_dnj6bc6,1506435869,c_d26,t3_72kg2a,t3_72kg2a,My boyfriend and I are the same ages as you an...,26.0,AmItheAsshole,NaN,304.0
4,427576402,t3_72kg2a,1506433689,1.0,0.0,Ritsku,AITA for breaking up with my girlfriend becaus...,NaN,My girlfriend recently went to the beach with ...,679.0,...,t1_dnj6qcs,1506436350,[deleted],t3_72kg2a,t3_72kg2a,"You are completely right, if she does somethin...",31.0,AmItheAsshole,NaN,120.0


Now, let's iterate over our merged DataFrame and group all associated submissions and comments together. 
- We first create an empty dictionary.
- We then iterate over our DataFrame using the Pandas `.iterrows()` method, which allows us to iterate over rows in a DataFrame in a for-loop. It yields a tuple consisting of the index and the row.
- `If` the "idstr_x" column does not yet exist in the keys of our new dictionary, add it. As the value, add the "selftext" (i.e., the original post) and "body" (i.e., the first comment) columns.
- `Else`, if we have the key already, only add the "body" column (i.e., the comment). 

In [28]:
data_d = {}
for i, r in df_t.iterrows():
    if r.idstr_x not in data_d.keys():
        data_d[r.idstr_x] = [r.selftext, r.body]    
    else:
        data_d[r.idstr_x].append(r.body)    

In [30]:
# See if it works
data_d['t3_72kg2a']

['My girlfriend recently went to the beach with a few of her friends.  She has this tiny bikini bottom that is basically a thong that I HATE when she wears in public.  Well she wore it.  Not only did she wear it, she posed in the bathroom mirror of her hotel room to take a side profile picture so you could see her ass sticking out in it and posted it to her Snapchat story.   Worth mentioning I am not friends with her on Snapchat for reasons similar to this (sick of getting in fights when she says she\'s going out for \'girls\' night then posts videos of her sitting at a table with like 5 dudes that always got invited by one of the other girls which was completely unknown to her until she arrived - most of these guys she then adds on Snapchat afterwards).  She didn\'t even save it and send it to me.  I saw it when she was showing me pics from her beach trip and she had screenshot that particular snap and left it in her camera roll.  Whether the ass part was intentional or not I will nev

Finally, we'll join the items in each of the values in our `dict` and put that in a list. Each item will contain the post and its associated comments.

In [31]:
threads = [' '.join(thread) for thread in data_d.values()]
len(threads)

936

In [32]:
threads[0]

'My girlfriend recently went to the beach with a few of her friends.  She has this tiny bikini bottom that is basically a thong that I HATE when she wears in public.  Well she wore it.  Not only did she wear it, she posed in the bathroom mirror of her hotel room to take a side profile picture so you could see her ass sticking out in it and posted it to her Snapchat story.   Worth mentioning I am not friends with her on Snapchat for reasons similar to this (sick of getting in fights when she says she\'s going out for \'girls\' night then posts videos of her sitting at a table with like 5 dudes that always got invited by one of the other girls which was completely unknown to her until she arrived - most of these guys she then adds on Snapchat afterwards).  She didn\'t even save it and send it to me.  I saw it when she was showing me pics from her beach trip and she had screenshot that particular snap and left it in her camera roll.  Whether the ass part was intentional or not I will neve

Let's now save this text to a new file. 

In [34]:
with open('amita_threads.txt', 'w', encoding='utf_8') as f:
    for t in threads:
        f.write(t)